### Result Tests

A test strategy base on evidence shown above. Invest current 6:00 to 6:30 cryptos due to return in the same time slot on yesterday, do a reversal investment. If return of 6:30 on yesterday is negative, we short at 6:00 and long at 6:30 today.

Let's invest FIL at fix position of 10.

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [3]:
df = pd.read_csv('data/binance_30m_data.csv', index_col='timestamp', parse_dates=True)
df_returns = pd.read_csv('data/returns_data.csv', index_col='timestamp', parse_dates=True)

In [4]:
df.head()

,BTC_close,BTC_volume,ETH_close,ETH_volume,DOGE_close,DOGE_volume,SOL_close,SOL_volume,FIL_close,FIL_volume,LTC_close,LTC_volume,BNB_close,BNB_volume,SHIB_close,SHIB_volume,XRP_close,XRP_volume,AAVE_close,AAVE_volume
timestamp,,,,,,,,,,,,,,,,,,,,
2022-01-01 00:00:00,47379.13,793.08192,3750.69,5929.8941,0.1720,13752546.0,173.71,15366.34,34.84,66770.35,148.8,6934.841,520.3,12504.491,0.000034,7.312235e+10,0.8439,3060673.0,262.9,2787.897
2022-01-01 00:30:00,47192.55,607.65450,3735.61,3423.3115,0.1717,6722588.0,173.26,13494.15,34.77,57637.66,148.1,7045.217,518.0,10650.522,0.000034,7.080380e+10,0.8421,3106081.0,261.3,3476.485
2022-01-01 01:00:00,47130.36,267.90481,3727.65,2674.4177,0.1716,2736656.0,172.61,16990.42,34.79,23737.43,147.7,3747.091,516.3,14248.945,0.000034,8.492489e+10,0.8415,2017950.0,260.8,1057.201
2022-01-01 01:30:00,46979.62,346.07295,3711.92,2564.5849,0.1711,5697811.0,172.45,13714.25,34.62,28214.49,147.1,5129.352,515.1,8228.759,0.000034,5.998779e+10,0.8377,7761027.0,259.2,1126.356
2022-01-01 02:00:00,46959.61,295.25750,3707.96,3287.8317,0.1710,4065516.0,172.40,7702.92,34.69,29227.32,147.2,7548.368,513.8,11165.965,0.000034,6.331962e+10,0.8382,2493952.0,258.4,1561.486


In [5]:
df_fil =df[['FIL_close']]
df_fil_ret_ = df_returns[['FIL_return']]

In [8]:
df_fil['FIL_open'] = df_fil['FIL_close'].shift()

In [9]:
df_fil = pd.merge(df_fil, df_fil_ret_, left_index=True, right_index=True)

In [10]:
# calculate yesterday's return
df_fil['FIL_lag1d_ret'] = df_fil['FIL_return'].shift(48)
df_fil = df_fil.dropna()

In [11]:
df_fil.head()

,FIL_close,FIL_open,FIL_return,FIL_lag1d_ret
timestamp,,,,
2022-01-02 00:30:00,35.51,35.45,0.001693,-0.002009
2022-01-02 01:00:00,35.47,35.51,-0.001126,0.000575
2022-01-02 01:30:00,35.37,35.47,-0.002819,-0.004886
2022-01-02 02:00:00,35.44,35.37,0.001979,0.002022
2022-01-02 02:30:00,35.41,35.44,-0.000847,0.004036


In [12]:
df_fil['FIL_profit'] = np.where(df_fil['FIL_lag1d_ret'] < 0, # if negative returns
                                (df_fil['FIL_close'] - df_fil['FIL_open']) * 10, # we short open and long close to earn profit
                                (df_fil['FIL_open'] - df_fil['FIL_close']) * 10) # instead, we long open and short close
# calculate the profit if we buy open sell close all the time
df_fil['FIL_profit_r'] = (df_fil['FIL_close'] - df_fil['FIL_open']) * 10
# sum of profit
print('Profit via strategy: ', df_fil.FIL_profit.sum())
print('Profit buy open sell close: ', df_fil.FIL_profit_r.sum())

Profit via strategy:  166.64000000000036
Profit buy open sell close:  -324.24


In [13]:
# invest $1000 in the beginning, calculate the profit at the end of the year
position = 1000 / df_fil.iloc[0, 0]
df_fil['FIL_profit_f'] = np.where(df_fil['FIL_lag1d_ret'] < 0, # if negative returns
                                (df_fil['FIL_close'] - df_fil['FIL_open']) * position, # we short open and long close to earn profit
                                (df_fil['FIL_open'] - df_fil['FIL_close']) * position) # instead, we long open and short close
print('Profit if invest $1000: ', df_fil.FIL_profit_f.sum())

Profit if invest $1000:  469.2762602083929


#### What about Bitcoin

In [14]:
df_btc =df[['BTC_close']]
df_btc_ret_ = df_returns[['BTC_return']]
df_btc['BTC_open'] = df_btc['BTC_close'].shift()
df_btc = pd.merge(df_btc, df_btc_ret_, left_index=True, right_index=True)

In [15]:
# calculate yesterday's return
df_btc['BTC_lag1d_ret'] = df_btc['BTC_return'].shift(48)
df_btc = df_btc.dropna()

In [16]:
df_btc.head()

,BTC_close,BTC_open,BTC_return,BTC_lag1d_ret
timestamp,,,,
2022-01-02 00:30:00,47081.28,46945.27,0.002897,-0.003938
2022-01-02 01:00:00,47106.80,47081.28,0.000542,-0.001318
2022-01-02 01:30:00,47122.61,47106.80,0.000336,-0.003198
2022-01-02 02:00:00,47181.44,47122.61,0.001248,-0.000426
2022-01-02 02:30:00,47173.45,47181.44,-0.000169,0.005007


In [17]:
df_btc['BTC_profit'] = np.where(df_btc['BTC_lag1d_ret'] < 0, # if negative returns
                                (df_btc['BTC_close'] - df_btc['BTC_open']) * 10, # we short open and long close to earn profit
                                (df_btc['BTC_open'] - df_btc['BTC_close']) * 10) # instead, we long open and short close
# calculate the profit if we buy open sell close all the time
df_btc['BTC_profit_r'] = (df_btc['BTC_close'] - df_btc['BTC_open']) * 10

print('Profit via strategy: ', df_btc.BTC_profit.sum())
print('Profit buy open sell close: ', df_btc.BTC_profit_r.sum())

Profit via strategy:  174617.60000000361
Profit buy open sell close:  -304242.0


In [18]:
# invest $1000 in the beginning, calculate the profit at the end of the year
position = 1000 / df_btc.iloc[0, 0]
df_btc['BTC_profit_f'] = np.where(df_btc['BTC_lag1d_ret'] < 0, # if negative returns
                                (df_btc['BTC_close'] - df_btc['BTC_open']) * position, # we short open and long close to earn profit
                                (df_btc['BTC_open'] - df_btc['BTC_close']) * position) # instead, we long open and short close
print('Profit if invest $1000: ', df_btc.BTC_profit_f.sum())

Profit if invest $1000:  370.8854134806947


#### Apply same calculation for all coins

#### ETH

In [19]:
df_eth =df[['ETH_close']]
df_eth_ret_ = df_returns[['ETH_return']]
df_eth['ETH_open'] = df_eth['ETH_close'].shift()
df_eth = pd.merge(df_eth, df_eth_ret_, left_index=True, right_index=True)
# calculate yesterday's return
df_eth['ETH_lag1d_ret'] = df_eth['ETH_return'].shift(48)
df_eth = df_eth.dropna()
df_eth['ETH_profit'] = np.where(df_eth['ETH_lag1d_ret'] < 0, # if negative returns
                                (df_eth['ETH_close'] - df_eth['ETH_open']) * 10, # we short open and long close to earn profit
                                (df_eth['ETH_open'] - df_eth['ETH_close']) * 10) # instead, we long open and short close
# calculate the profit if we buy open sell close all the time
df_eth['ETH_profit_r'] = (df_eth['ETH_close'] - df_eth['ETH_open']) * 10

print('Profit via strategy: ', df_eth.ETH_profit.sum())
print('Profit buy open sell close: ', df_eth.ETH_profit_r.sum())

Profit via strategy:  32726.199999999943
Profit buy open sell close:  -25359.399999999994


In [20]:
# invest $1000 in the beginning, calculate the profit at the end of the year
position = 1000 / df_eth.iloc[0, 0]
df_eth['ETH_profit_f'] = np.where(df_eth['ETH_lag1d_ret'] < 0, # if negative returns
                                (df_eth['ETH_close'] - df_eth['ETH_open']) * position, # we short open and long close to earn profit
                                (df_eth['ETH_open'] - df_eth['ETH_close']) * position) # instead, we long open and short close
print('Profit if invest $1000: ', df_eth.ETH_profit_f.sum())

Profit if invest $1000:  873.9361602696071


#### DOGE

In [21]:
df_doge =df[['DOGE_close']]
df_doge_ret_ = df_returns[['DOGE_return']]
df_doge['DOGE_open'] = df_doge['DOGE_close'].shift()
df_doge = pd.merge(df_doge, df_doge_ret_, left_index=True, right_index=True)
# calculate yesterday's return
df_doge['DOGE_lag1d_ret'] = df_doge['DOGE_return'].shift(48)
df_doge = df_doge.dropna()
df_doge['DOGE_profit'] = np.where(df_doge['DOGE_lag1d_ret'] < 0, # if negative returns
                                (df_doge['DOGE_close'] - df_doge['DOGE_open']) * 10, # we short open and long close to earn profit
                                (df_doge['DOGE_open'] - df_doge['DOGE_close']) * 10) # instead, we long open and short close
# calculate the profit if we buy open sell close all the time
df_doge['DOGE_profit_r'] = (df_doge['DOGE_close'] - df_doge['DOGE_open']) * 10

print('Profit via strategy: ', df_doge.DOGE_profit.sum())
print('Profit buy open sell close: ', df_doge.DOGE_profit_r.sum())

Profit via strategy:  4.8559999999999945
Profit buy open sell close:  -1.018


In [22]:
# invest $1000 in the beginning, calculate the profit at the end of the year
position = 1000 / df_doge.iloc[0, 0]
df_doge['DOGE_profit_f'] = np.where(df_doge['DOGE_lag1d_ret'] < 0, # if negative returns
                                (df_doge['DOGE_close'] - df_doge['DOGE_open']) * position, # we short open and long close to earn profit
                                (df_doge['DOGE_open'] - df_doge['DOGE_close']) * position) # instead, we long open and short close
print('Profit if invest $1000: ', df_doge.DOGE_profit_f.sum())

Profit if invest $1000:  2828.1887012230613


#### SOL

In [23]:
df_sol =df[['SOL_close']]
df_sol_ret_ = df_returns[['SOL_return']]
df_sol['SOL_open'] = df_sol['SOL_close'].shift()
df_sol = pd.merge(df_sol, df_sol_ret_, left_index=True, right_index=True)
# calculate yesterday's return
df_sol['SOL_lag1d_ret'] = df_sol['SOL_return'].shift(48)
df_sol = df_sol.dropna()
df_sol['SOL_profit'] = np.where(df_sol['SOL_lag1d_ret'] < 0, # if negative returns
                                (df_sol['SOL_close'] - df_sol['SOL_open']) * 10, # we short open and long close to earn profit
                                (df_sol['SOL_open'] - df_sol['SOL_close']) * 10) # instead, we long open and short close
# calculate the profit if we buy open sell close all the time
df_sol['SOL_profit_r'] = (df_sol['SOL_close'] - df_sol['SOL_open']) * 10

print('Profit via strategy: ', df_sol.SOL_profit.sum())
print('Profit buy open sell close: ', df_sol.SOL_profit_r.sum())

Profit via strategy:  2144.9999999999995
Profit buy open sell close:  -1665.2


In [24]:
# invest $1000 in the beginning, calculate the profit at the end of the year
position = 1000 / df_sol.iloc[0, 0]
df_sol['SOL_profit_f'] = np.where(df_sol['SOL_lag1d_ret'] < 0, # if negative returns
                                (df_sol['SOL_close'] - df_sol['SOL_open']) * position, # we short open and long close to earn profit
                                (df_sol['SOL_open'] - df_sol['SOL_close']) * position) # instead, we long open and short close
print('Profit if invest $1000: ', df_sol.SOL_profit_f.sum())

Profit if invest $1000:  1214.334239130435


#### LTC

In [25]:
df_ltc =df[['LTC_close']]
df_ltc_ret_ = df_returns[['LTC_return']]
df_ltc['LTC_open'] = df_ltc['LTC_close'].shift()
df_ltc = pd.merge(df_ltc, df_ltc_ret_, left_index=True, right_index=True)
# calculate yesterday's return
df_ltc['LTC_lag1d_ret'] = df_ltc['LTC_return'].shift(48)
df_ltc = df_ltc.dropna()
df_ltc['LTC_profit'] = np.where(df_ltc['LTC_lag1d_ret'] < 0, # if negative returns
                                (df_ltc['LTC_close'] - df_ltc['LTC_open']) * 10, # we short open and long close to earn profit
                                (df_ltc['LTC_open'] - df_ltc['LTC_close']) * 10) # instead, we long open and short close
# calculate the profit if we buy open sell close all the time
df_ltc['LTC_profit_r'] = (df_ltc['LTC_close'] - df_ltc['LTC_open']) * 10

print('Profit via strategy: ', df_ltc.LTC_profit.sum())
print('Profit buy open sell close: ', df_ltc.LTC_profit_r.sum())

Profit via strategy:  1825.1999999999985
Profit buy open sell close:  -795.8


In [26]:
# invest $1000 in the beginning, calculate the profit at the end of the year
position = 1000 / df_ltc.iloc[0, 0]
df_ltc['LTC_profit_f'] = np.where(df_ltc['LTC_lag1d_ret'] < 0, # if negative returns
                                (df_ltc['LTC_close'] - df_ltc['LTC_open']) * position, # we short open and long close to earn profit
                                (df_ltc['LTC_open'] - df_ltc['LTC_close']) * position) # instead, we long open and short close
print('Profit if invest $1000: ', df_ltc.LTC_profit_f.sum())

Profit if invest $1000:  1222.5050234427317


#### BNB

In [27]:
df_bnb =df[['BNB_close']]
df_bnb_ret_ = df_returns[['BNB_return']]
df_bnb['BNB_open'] = df_bnb['BNB_close'].shift()
df_bnb = pd.merge(df_bnb, df_bnb_ret_, left_index=True, right_index=True)
# calculate yesterday's return
df_bnb['BNB_lag1d_ret'] = df_bnb['BNB_return'].shift(48)
df_bnb = df_bnb.dropna()
df_bnb['BNB_profit'] = np.where(df_bnb['BNB_lag1d_ret'] < 0, # if negative returns
                                (df_bnb['BNB_close'] - df_bnb['BNB_open']) * 10, # we short open and long close to earn profit
                                (df_bnb['BNB_open'] - df_bnb['BNB_close']) * 10) # instead, we long open and short close
# calculate the profit if we buy open sell close all the time
df_bnb['BNB_profit_r'] = (df_bnb['BNB_close'] - df_bnb['BNB_open']) * 10

print('Profit via strategy: ', df_bnb.BNB_profit.sum())
print('Profit buy open sell close: ', df_bnb.BNB_profit_r.sum())

Profit via strategy:  5643.999999999951
Profit buy open sell close:  -2772.0


In [28]:
# invest $1000 in the beginning, calculate the profit at the end of the year
position = 1000 / df_bnb.iloc[0, 0]
df_bnb['BNB_profit_f'] = np.where(df_bnb['BNB_lag1d_ret'] < 0, # if negative returns
                                (df_bnb['BNB_close'] - df_bnb['BNB_open']) * position, # we short open and long close to earn profit
                                (df_bnb['BNB_open'] - df_bnb['BNB_close']) * position) # instead, we long open and short close
print('Profit if invest $1000: ', df_bnb.BNB_profit_f.sum())

Profit if invest $1000:  1079.3650793650702


#### SHIB

In [29]:
df_shib =df[['SHIB_close']]
df_shib_ret_ = df_returns[['SHIB_return']]
df_shib['SHIB_open'] = df_shib['SHIB_close'].shift()
df_shib = pd.merge(df_shib, df_shib_ret_, left_index=True, right_index=True)
# calculate yesterday's return
df_shib['SHIB_lag1d_ret'] = df_shib['SHIB_return'].shift(48)
df_shib = df_shib.dropna()
df_shib['SHIB_profit'] = np.where(df_shib['SHIB_lag1d_ret'] < 0, # if negative returns
                                (df_shib['SHIB_close'] - df_shib['SHIB_open']) * 10, # we short open and long close to earn profit
                                (df_shib['SHIB_open'] - df_shib['SHIB_close']) * 10) # instead, we long open and short close
# calculate the profit if we buy open sell close all the time
df_shib['SHIB_profit_r'] = (df_shib['SHIB_close'] - df_shib['SHIB_open']) * 10

print('Profit via strategy: ', df_shib.SHIB_profit.sum())
print('Profit buy open sell close: ', df_shib.SHIB_profit_r.sum())

Profit via strategy:  0.001062599999999999
Profit buy open sell close:  -0.000256


In [30]:
# invest $1000 in the beginning, calculate the profit at the end of the year
position = 1000 / df_shib.iloc[0, 0]
df_shib['SHIB_profit_f'] = np.where(df_shib['SHIB_lag1d_ret'] < 0, # if negative returns
                                (df_shib['SHIB_close'] - df_shib['SHIB_open']) * position, # we short open and long close to earn profit
                                (df_shib['SHIB_open'] - df_shib['SHIB_close']) * position) # instead, we long open and short close
print('Profit if invest $1000: ', df_shib.SHIB_profit_f.sum())

Profit if invest $1000:  3151.24555160142


#### XRP

In [31]:
df_xrp =df[['XRP_close']]
df_xrp_ret_ = df_returns[['XRP_return']]
df_xrp['XRP_open'] = df_xrp['XRP_close'].shift()
df_xrp = pd.merge(df_xrp, df_xrp_ret_, left_index=True, right_index=True)
# calculate yesterday's return
df_xrp['XRP_lag1d_ret'] = df_xrp['XRP_return'].shift(48)
df_xrp = df_xrp.dropna()
df_xrp['XRP_profit'] = np.where(df_xrp['XRP_lag1d_ret'] < 0, # if negative returns
                                (df_xrp['XRP_close'] - df_xrp['XRP_open']) * 10, # we short open and long close to earn profit
                                (df_xrp['XRP_open'] - df_xrp['XRP_close']) * 10) # instead, we long open and short close
# calculate the profit if we buy open sell close all the time
df_xrp['XRP_profit_r'] = (df_xrp['XRP_close'] - df_xrp['XRP_open']) * 10

print('Profit via strategy: ', df_xrp.XRP_profit.sum())
print('Profit buy open sell close: ', df_xrp.XRP_profit_r.sum())

Profit via strategy:  7.303999999999936
Profit buy open sell close:  -5.039999999999999


In [32]:
# invest $1000 in the beginning, calculate the profit at the end of the year
position = 1000 / df_xrp.iloc[0, 0]
df_xrp['XRP_profit_f'] = np.where(df_xrp['XRP_lag1d_ret'] < 0, # if negative returns
                                (df_xrp['XRP_close'] - df_xrp['XRP_open']) * position, # we short open and long close to earn profit
                                (df_xrp['XRP_open'] - df_xrp['XRP_close']) * position) # instead, we long open and short close
print('Profit if invest $1000: ', df_xrp.XRP_profit_f.sum())

Profit if invest $1000:  866.2239089183986


#### AAVE

In [33]:
df_aave =df[['AAVE_close']]
df_aave_ret_ = df_returns[['AAVE_return']]
df_aave['AAVE_open'] = df_aave['AAVE_close'].shift()
df_aave = pd.merge(df_aave, df_aave_ret_, left_index=True, right_index=True)
# calculate yesterday's return
df_aave['AAVE_lag1d_ret'] = df_aave['AAVE_return'].shift(48)
df_aave = df_aave.dropna()
df_aave['AAVE_profit'] = np.where(df_aave['AAVE_lag1d_ret'] < 0, # if negative returns
                                (df_aave['AAVE_close'] - df_aave['AAVE_open']) * 10, # we short open and long close to earn profit
                                (df_aave['AAVE_open'] - df_aave['AAVE_close']) * 10) # instead, we long open and short close
# calculate the profit if we buy open sell close all the time
df_aave['AAVE_profit_r'] = (df_aave['AAVE_close'] - df_aave['AAVE_open']) * 10

print('Profit via strategy: ', df_aave.AAVE_profit.sum())
print('Profit buy open sell close: ', df_aave.AAVE_profit_r.sum())

Profit via strategy:  2198.9999999999877
Profit buy open sell close:  -2055.0


In [34]:
# invest $1000 in the beginning, calculate the profit at the end of the year
position = 1000 / df_aave.iloc[0, 0]
df_aave['AAVE_profit_f'] = np.where(df_aave['AAVE_lag1d_ret'] < 0, # if negative returns
                                (df_aave['AAVE_close'] - df_aave['AAVE_open']) * position, # we short open and long close to earn profit
                                (df_aave['AAVE_open'] - df_aave['AAVE_close']) * position) # instead, we long open and short close
print('Profit if invest $1000: ', df_aave.AAVE_profit_f.sum())

Profit if invest $1000:  863.0298273155369
